In [ ]:
%%html
<script>
    var code_show=true; //true -> hide code at first

    function code_toggle() {
        $('div.prompt').hide(); // always hide prompt

        if (code_show){
            $('div.input').hide();
        } else {
            $('div.input').show();
        }
        code_show = !code_show
    }
    $( document ).ready(code_toggle);
</script>
<p style="font-size:60%;">
<a href="javascript:code_toggle()">[Toggle Code]</a>
<a target="_blank" href="./../project/project_set_params_gui.ipynb#">[Set params]</a>
</p>

In [ ]:
%matplotlib inline
import os
import re
import sys
from openquake.mbt.oqt_project import OQtProject
from openquake.mbt.tools.faults import rates_for_double_truncated_mfd
from openquake.hazardlib.geo.surface import SimpleFaultSurface
from openquake.hazardlib.mfd import EvenlyDiscretizedMFD
from openquake.hazardlib.scalerel.wc1994 import WC1994

In [ ]:
project_pickle_filename = os.environ.get('OQMBT_PROJECT')
oqtkp = OQtProject.load_from_file(project_pickle_filename)
model_id = oqtkp.active_model_id
model = oqtkp.models[model_id]
try:
    source_ids_list = getattr(oqtkp,'active_source_id')
except:
    print('Active source ID not defined in the OQMBT project')
    source_ids_list = ['sf1']
print('Processing fault source with ID:', source_ids_list)

## Parameters

In [ ]:
default_mmin = float(model.faults_lower_threshold_magnitude)
bin_width = float(model.mfd_binwidth)
scalerel_name = model.msr

## Set the magnitude scaling relationship
The options available for the magnitude scaling relationship are:
* `scalrel_name = 'WC1994`' <br> Wells and Coppersmith (1994) scaling relationship
* '`PointMSR`' Scaling relationship to be used for the creation of tiny ruptures resembling points
* '`CEUS2011`' Scaling relationship developed within the CEUS SSC project

In [ ]:
import importlib
module = importlib.import_module('openquake.hazardlib.scalerel')
my_class = getattr(module, scalerel_name)
scalrel = my_class()

## Computing the mfd

In [ ]:
assert len(source_ids_list) == 1
src = model.sources[source_ids_list[0]]

if hasattr(src, 'mlow') and src.mlow >= 0.0:
    m_low = src.mlow
else:
    m_low = 0.0

if hasattr(src, 'mmin') and src.mmin > 0.0:
    m_min = src.mmin
else:
    m_min = default_mmin

for key in src.__dict__:
    print(key, getattr(src, key))

In [ ]:
if src.source_type == 'SimpleFaultSource':
    
    if True:
        
        # Fix upper seismogenic depth
        if hasattr(src, 'upper_seismogenic_depth'):
            upper_seismogenic_depth = 0
        else:
            upper_seismogenic_depth = src.upper_seismogenic_depth
            
        # Scaling relationship
        if hasattr(src, 'scalerel'):
            scalerel = getattr(src, 'scalerel')
        else:
            scalerel = WC1994()

        # Compute area source
        surf = SimpleFaultSurface.from_fault_data(
                    fault_trace = src.trace, 
                    upper_seismogenic_depth = upper_seismogenic_depth, 
                    lower_seismogenic_depth = src.lower_seismogenic_depth, 
                    dip = src.dip, 
                    mesh_spacing = 2.5)
        flt_area = surf.get_area()

        # Median magnitude
        if hasattr(src, 'mmax') and float(getattr(src, 'mmax')) > 0.0:
            mmax = float(getattr(src, 'mmax'))
            print('Maximum magnitude (from input)               : %.2f' % (mmax))
        else:
            median_magnitude = scalrel.get_median_mag(flt_area, src.rake)
            std = scalrel.get_std_dev_mag(src.rake)
            mmax = median_magnitude + std * 2
            print('Maximum magnitude (from scaling relationship): %.2f' % (mmax))
            print('Magnitude Standard Deviation                 : %.2f' % (std))
         
        if mmax > m_min+bin_width and src.slip_rate > 1e-10:
        
            # Get rates
            if hasattr(src, 'id_intersecting_area'):
                print('Using parameters of area source ID           : %s' % (src.id_intersecting_area))
                b_gr = model.sources[src.id_intersecting_area].b_gr
            else:
                b_gr = model.default_bgr
            
            if hasattr(src, 'coeff'):
                coeff = float(getattr(src, 'coeff'))
                print('Coeff parameter (from input)                 : %.2f' % (coeff))
            else:
                coeff = 1.0
                print('Coeff parameter (default)                    : %.2f' % (coeff))
            
            rates = rates_for_double_truncated_mfd(flt_area, 
                                                   src.slip_rate*coeff, 
                                                   m_low, 
                                                   mmax,
                                                   m_min,
                                                   b_gr=b_gr,
                                                   bin_width=bin_width)
            
            if any(rates):
                src.mfd = EvenlyDiscretizedMFD(m_min+bin_width/2, bin_width, rates)
                src.scalerel = scalrel
                print('Computed MFD\n\n',src.mfd.get_annual_occurrence_rates())
            else:
                print('MFD not defined: rates not defined')
                src.mfd = None
                src.scalerel = None                
        else:  
            print('MFD not defined: mag from scaling relationship < m_min + bin_width')
            src.mfd = None
            src.scalerel = None

model.sources[source_ids_list[0]] = src 
oqtkp.models[model_id] = model
oqtkp.save()